#資料載入
將從kaggle下載的[新聞資料集](https://www.kaggle.com/datasets/nikhiljohnk/news-popularity-in-multiple-social-media-platforms)，下載的資料放入colab /content內

In [5]:
import csv

In [6]:
# 將csv檔案載入
news_arr = []
with open("train_file.csv","r") as cs:
  tmp = csv.reader(cs)
  for row in tmp:
    news_arr.append(row)


In [7]:
# 環境初始化
! wget -O init_env.sh https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0 && \
bash init_env.sh
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.9-src.zip")
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

--2022-03-31 07:49:29--  https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh [following]
--2022-03-31 07:49:29--  https://www.dropbox.com/s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1a92f819c83da9aec13e992e36.dl.dropboxusercontent.com/cd/0/inline/BidFFWLd1vXX3XOhmSfptZa7A81Dw1475b-fra4f-UqCbXzvOPgwKDNI7C9iMWL_q_wYtbLZqxHbtrMSiuykZIdRAhaBengSNbr2DZJhk7t58mdeKZzx_3DiQhX6JiNMHdKUZD9i8gnGzppj6fp1YXysiBoX46ZGUbaHFDIWUPwg2A/file# [following]
--2022-03-31 07:49:30--  https://uc1a92f819c83da9aec13e992e36.dl.dropboxusercontent.com/cd/0/inline/BidFFWLd1vXX3XOhmS

In [8]:
# 將資料放入spark內
news_data = sc.parallelize(news_arr)
# 檢查資料數量
news_data.count()

55933

In [9]:
# 資料觀察
news_data.take(10)

[['IDLink',
  'Title',
  'Headline',
  'Source',
  'Topic',
  'PublishDate',
  'Facebook',
  'GooglePlus',
  'LinkedIn',
  'SentimentTitle',
  'SentimentHeadline'],
 ['Tr3CMgRv1N',
  'Obama Lays Wreath at Arlington National Cemetery',
  'Obama Lays Wreath at Arlington National Cemetery. President Barack Obama has laid a wreath at the Tomb of the Unknowns to honor',
  'USA TODAY',
  'obama',
  '2002-04-02 00:00:00',
  '-1',
  '-1',
  '-1',
  '0.0',
  '-0.0533001790889026'],
 ['Wc81vGp8qZ',
  'A Look at the Health of the Chinese Economy',
  'Tim Haywood, investment director business-unit head for fixed income at Gam, discusses the China beige book and the state of the economy.',
  'Bloomberg',
  'economy',
  '2008-09-20 00:00:00',
  '-1',
  '-1',
  '-1',
  '0.208333333333333',
  '-0.156385810542806'],
 ['zNGH03CrZH',
  'Nouriel Roubini: Global Economy Not Back to 2008',
  "Nouriel Roubini, NYU professor and chairman at Roubini Global Economics, explains why the global economy isn't facin

In [10]:
# 將需要的資料取出來
news_trim = news_data.map(lambda x : [x[4],x[2]])
# 比對資料數量
news_trim.count()

55933

In [11]:
# 觀察取出來的資料
news_trim.take(10)

[['Topic', 'Headline'],
 ['obama',
  'Obama Lays Wreath at Arlington National Cemetery. President Barack Obama has laid a wreath at the Tomb of the Unknowns to honor'],
 ['economy',
  'Tim Haywood, investment director business-unit head for fixed income at Gam, discusses the China beige book and the state of the economy.'],
 ['economy',
  "Nouriel Roubini, NYU professor and chairman at Roubini Global Economics, explains why the global economy isn't facing the same conditions"],
 ['economy',
  "Finland's economy expanded marginally in the three months ended December, after contracting in the previous quarter, preliminary figures from Statistics Finland showed Monday. "],
 ['economy',
  'Tourism and public spending continued to boost the economy in January, in light of contraction in private consumption and exports, according to the Bank of Thailand data. '],
 ['microsoft',
  'Over 100 attendees expected to see latest version of Microsoft Dynamics SL and Dynamics GP (PRWeb February 29, 2

In [12]:
!pip install -q transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
# 載入QA模型
model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
# 測試用QA模型做預測
QA_input = {
    'question': 'Which company develop AR headset?',
    'context': "Microsoft’s AR headset is being made available to developers, along with three new games – including one based on Conker’s Bad Fur Day."
}
res = nlp(QA_input)

     |████████████████████████████████| 3.8 MB 6.7 MB/s 
     |████████████████████████████████| 895 kB 61.2 MB/s 
     |████████████████████████████████| 6.5 MB 49.0 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 63.3 MB/s 


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [13]:
# 察看結果
res["answer"]

'Microsoft'

In [14]:
# 載入分詞套件
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [15]:
# 取出被分類成名詞的token
def get_noun(sentence):
  token = nltk.word_tokenize(sentence)
  token = nltk.pos_tag(token)
  res = []
  for i in token:
    if i[0].lower() == "news":
      continue
    if "NN" in i[1]:
      res.append(i[0])  
  return res

In [16]:
# 想要查詢的句子
sentence = 'News of Ukraine'
noun_list = get_noun(sentence)
print(noun_list)

['Ukraine']


In [17]:
# 如果有想要查詢句子的單字，才回傳
def have_word(sentence):
  for i in noun_list:
    if i in sentence[1]:
      return sentence
  else:
    return None

In [18]:
# 找回全部相關的文章丟入QAmodel
context = news_trim.map(lambda x : have_word(x)).filter(lambda y: y != None ).collect()
answers = []
for c in context:
  QA_input={'question': sentence, 'context':c[1]}
  answers.append(nlp(QA_input)["answer"])
  print(nlp(QA_input)["answer"])

sanctions
sanctions
sanctions
sanctions
Ukraine announced that beginning Friday, it would suspend its electricity imports
Ukraine’s economy probably exited 1 1/2 years of recession
Russia's Economic Development Minister
sanctions
Russia still expects Ukraine to pay debt
MOSCOW
draft law
conflict
worries over Russia's stopping gas supplies to Ukraine
political considerations
Ukraine tensions on sidelines of Paris climate talks
Ukraine
Moscow's annexation of Crimea
Syria
Crimea's authorities will file a lawsuit against Ukraine
SIMFEROPOL
weapons
Ukraine
KIEV
Russia regrets Ukraine has preferred default to talks
Ukraine
Russia
conflict in Ukraine
&quot;Starting from January 1, 2016
BRUSSELS
Coalminers
Ukraine for two years is now easing
Ukraine and Colombia
Ukraine cannot impose any counter sanctions against Russia
Ukraine is negotiating the creation of free trade zones with Turkey and Israel
Ukraine
Ukraine's Feb. 2014 revolution
Ukraine's Feb. 2014 revolution
Ukraine's Feb. 2014 revolut

In [19]:
# 將答案輸出
sc.parallelize(answers).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1]).collect()

[('Ukraine announced that beginning Friday, it would suspend its electricity imports',
  1),
 ('Ukraine’s economy probably exited 1 1/2 years of recession', 1),
 ("Russia's Economic Development Minister", 1),
 ("Moscow's annexation of Crimea", 1),
 ('SIMFEROPOL', 1),
 ('weapons', 1),
 ('Russia regrets Ukraine has preferred default to talks', 1),
 ('conflict in Ukraine', 1),
 ('&quot;Starting from January 1, 2016', 1),
 ('Coalminers', 1),
 ('Ukraine for two years is now easing', 1),
 ('Ukraine and Colombia', 1),
 ('Ukraine is negotiating the creation of free trade zones with Turkey and Israel',
  1),
 ('local economies are starting to fall apart.', 1),
 ('The EU and Ukraine', 1),
 ('Russian food products banned for importation into Ukraine from January 10.',
  1),
 ("Ukraine's Ministry of Economic Development and Trade", 1),
 ('Russian', 1),
 ('Ukrainian Economic Assessment is a comprehensive study quantifying economic damage from the current crisis',
  1),
 ('French Economy Minister Em